# Co-factor swap

Some text

In [1]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield, biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo import load_model

In [2]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [3]:
# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 


In [4]:
# Finding the optimal fluxes through each reaction when the objective is biomass production 
Fluxes = model.optimize()
Fluxes

,fluxes,reduced_costs
CITtcp,0.00000,0.000000
13BGH,0.00000,-0.047516
13BGHe,0.00000,0.000000
13GS,0.32667,0.000000
16GS,0.00000,0.000000
...,...,...
R03353,0.00000,0.000000
R07215,0.00000,0.000000
R04804,0.00000,0.000000
SK_methylpentanal_c,0.00000,0.000000


In [5]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c


In [6]:
# Setting the progesterone yield from glucose as substrate
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)
py

In [7]:
#optswap

In [8]:
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors swapped.
swap_1 = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nad_c","nadh_c"],["nadp_c","nadph_c"]),objective_function=py)

swap_1

In [9]:
# Identify the reactions that when swapped to another co-factor improve the production of progesterone
solution = swap_1.run(max_size = 20)

Starting optimization at Fri, 18 Nov 2022 15:10:08


HBox()

Finished after 00:03:44


In [10]:
solution

,index,targets,fitness
0,0,"(GAPD, LNS14DMx)",0.351464
1,1,"(GAPD,)",0.342857
2,5,"(ALCD2x_copy1,)",0.323077
3,6,"(ALCD2x_copy2,)",0.323077
4,12,"(MTHFD, MTHFD2i)",0.323077
5,23,"(MDH, ECYP11A1, R07215)",0.289655
6,24,"(GLUDy, C3STDH1, MDH, G3PD1ir)",0.271844
7,25,"(C4STMO2, MDH)",0.258461
8,26,"(R07498, HMGCOAR)",0.257668
9,27,"(MDH,)",0.253012


In [11]:
solution.model

Name,iMM904_progesterone
Memory address,0x07fde380c1310
Number of metabolites,1234
Number of reactions,1587
Number of groups,0
Objective expression,1.0*R02216 - 1.0*R02216_reverse_2ed70
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [12]:
solution.model

Name,iMM904_progesterone
Memory address,0x07fde380c1310
Number of metabolites,1234
Number of reactions,1587
Number of groups,0
Objective expression,1.0*R02216 - 1.0*R02216_reverse_2ed70
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [13]:
reactions=solution.data_frame.targets
reactions

0                       (GAPD, LNS14DMx)
1                                (GAPD,)
2                        (ALCD2x_copy1,)
3                        (ALCD2x_copy2,)
4                       (MTHFD, MTHFD2i)
5                (MDH, ECYP11A1, R07215)
6         (GLUDy, C3STDH1, MDH, G3PD1ir)
7                         (C4STMO2, MDH)
8                      (R07498, HMGCOAR)
9                                 (MDH,)
10           (C4STMO1, R05703, ECYP11A1)
11                   (C3STKR1, ECYP11A1)
12                           (ECYP11A1,)
13              (C3STKR1, C4STMO1, SQLS)
14            (C4STMO2, C3STDH2, R07215)
15            (R07215, C4STMO1, C3STKR2)
16               (C4STMO2, SQLS, R01456)
17    (C3STDH1, C3STKR1, C14STR, R07215)
18                            (C4STMO2,)
19                            (C4STMO1,)
20             (C3STKR1, R01456, R07215)
21             (C3STKR1, R05703, R02216)
22                        (SQLS, C14STR)
23                        (SQLS, R02216)
24              

In [14]:
for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()

Reactions:
Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

H+ + Lanosterol C30H50O + O2 O2 + Nicotinamide adenine dinucleotide - reduced ==> 4 4 dimethylcholesta 8 14 24 trienol C29H46O + Formate + H2O H2O + Nicotinamide adenine dinucleotide

Reactions:
Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
Ethanol + Nicotinamide adenine dinucleotide ==> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
Ethanol + Nicotinamide adenine dinucleotide ==> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
5,10-Methylenetetrahydrofolate + Nicotinamide adenine dinucleotide phosphate ==> 5,10-Methenyltetrahydrofolate + Nicotinamide adenine dinucleotide phosphate - reduced

5,10-Methylenetetrahydrofolate + Nicotina

In [15]:
py